In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from scripts.proj1_helpers import *
from scripts.implementations import *

In [3]:
# Import Data and methods
y_train_load, x_train_load, ids_train = load_csv_data('./data/train.csv', sub_sample=False)
_, x_test_load, ids_test = load_csv_data('./data/test.csv', sub_sample=False)

# Add 1 dimension FIXME not understood
y_train = np.expand_dims(y_train_load, axis=1)

# Logistic Regression Baseline

In [4]:
def compute_accuracy(y, y_prediction):
    correct = 0.0
    for i in range(len(y)):
        if y[i] == y_prediction[i]:
            correct += 1
    accuracy = correct/len(y)
    return accuracy

# replace -999 value by the mean of the corresponding feature without taking the aberrant values into account
def remove_aberrant(data):
    for i in range(data.shape[1]):
        data[:, i][data[:, i] == -999.0] = np.mean(data[:, i][data[:, i] != -999.0])

In [5]:
# build sampled x and y.
seed = 1
#y_train_samp, x_train_samp = sample_data(y_train, x_train, seed, size_samples=y_train_load.shape[0])

# standardize both sets
remove_aberrant(x_train_load)
remove_aberrant(x_test_load)
x_train, mean_x, std_x = standardize(x_train_load)
x_test, mean_test, std_test = standardize(x_test_load)

In [6]:
tx_train = np.c_[np.ones((y_train.shape[0], 1)), x_train]
tx_test = np.c_[np.ones((x_test.shape[0], 1)), x_test]

In [53]:
# logistic regression
initial_w = np.zeros((tx_train.shape[1], 1))
lambda_ = 600
gamma = 0.00001
#assert(lambda_ * gamma <= 1/2)
max_iters = 401

weights, loss = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iters, gamma)
#weights, loss = least_squares(y_train, tx_train)
print(loss)
# Sunday Morning todo:
# Problem exp overflow -> sigmoid is 1 -> log(0) -> nan
# Find reason for exp overflow? -> weights are not 0 anymore !!! -> huge number since 200'000 * weights
# this implies that:
# The accuracy I get is after a single update !!

Current iteration=0, loss=173286.79514000565
weights size:7.188392327298724


/home/yann/Documents/higgs-boson/scripts/implementations.py:200: RuntimeWarning: divide by zero encountered in log
  return np.squeeze(-(y.T @ np.log(sigmoid(tx @ w)) + (1 - y).T @ np.log(1 - sigmoid(tx @ w))))
/home/yann/Documents/higgs-boson/scripts/implementations.py:193: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))


KeyboardInterrupt: 

In [52]:
y_prediction = predict_labels(weights, tx_train)
print("Training set accuracy = "+str(compute_accuracy(y_train, y_prediction)))

Training set accuracy = 0.69864


## Helper methods

In [ ]:
def compute_accuracy(y, y_prediction):
    correct = 0.0
    for i in range(len(y)):
        if y[i] == y_prediction[i]:
            correct += 1
    accuracy = correct/len(y)
    return accuracy

# replace -999 value by the mean of the corresponding feature without taking the aberrant values into account
def remove_aberrant(data):
    for i in range(data.shape[1]):
        data[:, i][data[:, i] == -999.0] = np.mean(data[:, i][data[:, i] != -999.0])


In [ ]:
def cross_validation(y, x, k_fold, seed=1):
    """
    split the dataset in k equal subparts. Then train the model on k-1 parts and test it on the last remaining part.
    this process is repeatead k times with a different test part each time
    """
    assert(k_fold > 1)

    # set seed
    np.random.seed(seed)
    np.random.shuffle(x)
    # resetting the seed allows for an identical shuffling between y and x
    np.random.seed(seed)
    np.random.shuffle(y)
    
    loss_train = []
    loss_test = []
    acc_train, acc_test = [],[]
    
    total_length = len(y)
    split_length = int(total_length / k_fold)
    
    for i in range(k_fold):
        start_index = i * split_length
        if i == k_fold - 1:
            end_index = total_length
        else: #this case is to ensure that all elements are present, in case of unbalanced split
            end_index = (i + 1) * split_length
        
        tx_train = x[np.r_[0 : start_index, end_index: total_length]]
        y_train = y[np.r_[0 : start_index, end_index: total_length]]

        tx_test = x[start_index : end_index]
        y_test = y[start_index : end_index]

        # logistic regression
        initial_w = np.zeros((tx_train.shape[1], 1))
        lambda_ = 400
        gamma = 0.001
        assert(lambda_ * gamma <= 1/2)
        max_iters = 501

        weights, loss = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iters, gamma)
        y_train_pred = predict_labels(weights, tx_train)
        y_test_pred = predict_labels(weights, tx_test)
        
        loss_train.append(loss)
        loss_test.append(calculate_loss_logistic_regression(y_test, tx_test, weights))
        acc_train.append(compute_accuracy(y_train, y_train_pred))
        acc_test.append(compute_accuracy(y_test, y_test_pred))
        
    return np.mean(loss_train), np.mean(loss_test), np.mean(acc_train), np.mean(acc_test)

In [ ]:
cross_validation(y_train, tx_train, 4, seed)

In [ ]:
# Create submission
y_prediction_test = predict_labels(weights, tx_test)
create_csv_submission(ids_test, y_prediction_test, "submission.csv")
